In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import sklearn
from sklearn.model_selection import RandomizedSearchCV
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE

from avaml.aggregatedata import ForecastDataset, LabeledData, REG_ENG, CsvMissingError

# Introduction

We have a lot of input features and many labels. The goal of this Notebook is to see if we can trim the number of input features based on an array of different processing methods. For this Notebook, I will look at the following relationships to determine whether an input feature is relevant:
- correlations
- feature importances
- feature ranking with recursive feature elimination
- Principle Component Analysis (PCA)

Another interesting thing to look at would be to see what output features (labels) we can reasonably expect a machine learning model to predict. For instance, it may be that we can only predict half of the avlanche bulletin's output fields.

# Read in data

In [2]:
model_prefix = ''
days = 1
regobs_types = list(REG_ENG.keys())
labeled_data = None
try:
    print("Reading csv")
    labeled_data = LabeledData.from_csv(days=days, regobs_types=regobs_types, with_varsom=True)
except CsvMissingError:
    print("Csv missing. Fetching online data. (This takes a long time.)")
    labeled_data = ForecastDataset(regobs_types=regobs_types).label(days=days, with_varsom=True)
    labeled_data.to_csv()

Reading csv


# Data formatting, overall distributions

I go through some of these processing steps (with more printouts) in the Notebook on data distributions, so will not go through them here.

In [3]:
# first, drop regions
labeled_data.data = labeled_data.drop_regions().data

# then, rename indices
labeled_data.data = labeled_data.data.rename_axis(['date','region'])
labeled_data.label = labeled_data.label.rename_axis(['date', 'region'])

# flatten the hierchy of columns to 1D
labeled_data.data.columns = [' '.join(col).strip().replace(' ', '_') for col in labeled_data.data.columns.values]
labeled_data.label.columns = [' '.join(col).strip().replace(' ', '_') for col in labeled_data.label.columns.values]

# replace double underscores with single underscores
labeled_data.data.columns = [col.replace('__', '_') for col in labeled_data.data.columns.values]
labeled_data.label.columns = [col.replace('__', '_') for col in labeled_data.label.columns.values]

# convert some columns in labels to type int for averaging
labeled_data.label['CLASS_problem_amount'] = labeled_data.label['CLASS_problem_amount'].astype(int)
labeled_data.label['CLASS_danger_level'] = labeled_data.label['CLASS_danger_level'].astype(int)

for column in labeled_data.label.columns:
    if column.endswith(('_dist', '_dsize', '_lev_fill', '_prob', '_trig')):
        labeled_data.label[column] = labeled_data.label[column].astype(int)
        
# below, we can try to make categorical variables in the labels numeric
# first for the emergency warning column
warning_dict = {'Ikke gitt':0,
                'Naturlig utløste skred':1}

labeled_data.label['CLASS_emergency_warning'] = labeled_data.label['CLASS_emergency_warning'].replace(warning_dict)

# and now for the class problems
problem1 = list(np.unique(labeled_data.label.loc[:, 'CLASS_problem_1'].values))
problem2 = list(np.unique(labeled_data.label.loc[:, 'CLASS_problem_2'].values))
problem3 = list(np.unique(labeled_data.label.loc[:, 'CLASS_problem_3'].values))

list_of_problems = sorted(list(np.unique(problem1 + problem2 + problem3)))
problems_dict = {'':0, 'drift-slab':1, 'glide':2, 'new-loose':3,
                 'new-slab':4, 'pwl-slab':5, 'wet-loose':6, 'wet-slab':7}

labeled_data.label['CLASS_problem_1'] = labeled_data.label['CLASS_problem_1'].replace(problems_dict)
labeled_data.label['CLASS_problem_2'] = labeled_data.label['CLASS_problem_2'].replace(problems_dict)
labeled_data.label['CLASS_problem_3'] = labeled_data.label['CLASS_problem_3'].replace(problems_dict)

# Do we have redundant features that we can collapse?

In [10]:
for column in labeled_data.data.columns:
    if 'aspect' in column:
        print(column)

problem_new-loose_aspect_N_1
problem_new-loose_aspect_NE_1
problem_new-loose_aspect_E_1
problem_new-loose_aspect_SE_1
problem_new-loose_aspect_S_1
problem_new-loose_aspect_SW_1
problem_new-loose_aspect_W_1
problem_new-loose_aspect_NW_1
problem_wet-loose_aspect_N_1
problem_wet-loose_aspect_NE_1
problem_wet-loose_aspect_E_1
problem_wet-loose_aspect_SE_1
problem_wet-loose_aspect_S_1
problem_wet-loose_aspect_SW_1
problem_wet-loose_aspect_W_1
problem_wet-loose_aspect_NW_1
problem_new-slab_aspect_N_1
problem_new-slab_aspect_NE_1
problem_new-slab_aspect_E_1
problem_new-slab_aspect_SE_1
problem_new-slab_aspect_S_1
problem_new-slab_aspect_SW_1
problem_new-slab_aspect_W_1
problem_new-slab_aspect_NW_1
problem_drift-slab_aspect_N_1
problem_drift-slab_aspect_NE_1
problem_drift-slab_aspect_E_1
problem_drift-slab_aspect_SE_1
problem_drift-slab_aspect_S_1
problem_drift-slab_aspect_SW_1
problem_drift-slab_aspect_W_1
problem_drift-slab_aspect_NW_1
problem_pwl-slab_aspect_N_1
problem_pwl-slab_aspect_NE_1

In [11]:
for column in labeled_data.label.columns:
    print(labeled_data.label[column].dtypes, '     ', column)

int64       CLASS_danger_level
int64       CLASS_emergency_warning
int64       CLASS_problem_1
int64       CLASS_problem_2
int64       CLASS_problem_3
int64       CLASS_problem_amount
object       CLASS_drift-slab_cause
int64       CLASS_drift-slab_dist
int64       CLASS_drift-slab_dsize
int64       CLASS_drift-slab_lev_fill
int64       CLASS_drift-slab_prob
int64       CLASS_drift-slab_trig
object       CLASS_glide_cause
int64       CLASS_glide_dist
int64       CLASS_glide_dsize
int64       CLASS_glide_lev_fill
int64       CLASS_glide_prob
int64       CLASS_glide_trig
object       CLASS_new-loose_cause
int64       CLASS_new-loose_dist
int64       CLASS_new-loose_dsize
int64       CLASS_new-loose_lev_fill
int64       CLASS_new-loose_prob
int64       CLASS_new-loose_trig
object       CLASS_new-slab_cause
int64       CLASS_new-slab_dist
int64       CLASS_new-slab_dsize
int64       CLASS_new-slab_lev_fill
int64       CLASS_new-slab_prob
int64       CLASS_new-slab_trig
object       CLASS_p

In [12]:
look_at = ['MULTI_drift-slab_aspect', 'MULTI_glide_aspect', 'MULTI_new-loose_aspect', 
           'MULTI_new-slab_aspect', 'MULTI_pwl-slab_aspect', 'MULTI_wet-loose_aspect', 
           'MULTI_wet-slab_aspect']

labeled_data.label.loc[:, look_at]

MULTI_drift-slab_aspect MULTI_glide_aspect  \
date       region                                              
2017-11-23 3031                  11100011           00000000   
           3034                  11100011           00000000   
2017-12-02 3003                  11111111           00000000   
           3007                  00000000           00000000   
           3009                  00000000           00000000   
...                                   ...                ...   
2021-01-12 3031                  00000000           00000000   
           3032                  00000000           00000000   
           3034                  00000000           00000000   
           3035                  00000000           00000000   
           3037                  00000000           00000000   

                  MULTI_new-loose_aspect MULTI_new-slab_aspect  \
date       region                                                
2017-11-23 3031                 00000000              00000000   
           3034                 00000000              00000000   
2017-12-02 3003                 00000000              00000000   
           3007                 00000000              00000000   
           3009                 00000000              00000000   
...                                  ...                   ...   
2021-01-12 3031                 00000000              00000000   
           3032                 00000000              00000000   
           3034                 00000000              00000000   
           3035                 00000000              00000000   
           3037                 00000000              00000000   

                  MULTI_pwl-slab_aspect MULTI_wet-loose_aspect  \
date       region                                                
2017-11-23 3031                00000000               00000000   
           3034                00000000               00000000   
2017-12-02 3003                00000000               00000000   
           3007                11111111               00000000   
           3009                11111111               00000000   
...                                 ...                    ...   
2021-01-12 3031                11111111               00000000   
           3032                11111111               00000000   
           3034                11111111               00000000   
           3035                11111111               00000000   
           3037                11111111               00000000   

                  MULTI_wet-slab_aspect  
date       region                        
2017-11-23 3031                00000000  
           3034                00000000  
2017-12-02 3003                00000000  
           3007                00000000  
           3009                00000000  
...                                 ...  
2021-01-12 3031                00000000  
           3032                00000000  
           3034                00000000  
           3035                00000000  
           3037                00000000  

[13115 rows x 7 columns]

In [13]:
for i, row in enumerate(range(labeled_data.label.loc[:, look_at].replace('00000000', np.nan).shape[0])):
    print(labeled_data.label.loc[:, look_at].replace('00000000', np.nan).iloc[i, :].values)

['11100011' nan nan nan nan nan nan]
['11100011' nan nan nan nan nan nan]
['11111111' nan nan nan nan nan nan]
[nan nan nan nan '11111111' nan nan]
[nan nan nan nan '11111111' nan nan]
[nan nan nan nan '11111111' nan nan]
[nan nan nan nan '11111111' nan nan]
[nan nan nan nan '11001111' nan nan]
[nan nan nan nan '10001111' nan nan]
['11111111' nan nan nan nan nan nan]
['11100011' nan nan nan '11111111' nan nan]
['11100000' nan nan nan '11111000' nan nan]
[nan nan nan '11110001' '01111100' nan nan]
[nan nan nan nan '01111000' nan nan]
[nan nan nan nan '11110000' nan nan]
[nan nan nan nan '11110000' '11111111' nan]
[nan nan nan nan '11110000' '11111111' nan]
['01110000' nan nan nan '01111000' nan nan]
['01110000' nan nan nan '01111000' nan nan]
[nan nan nan nan '11110000' '11111111' nan]
['01110000' nan nan nan nan nan nan]
[nan nan nan nan '11111000' '11111111' nan]
['01110000' nan nan nan nan nan nan]
['11111111' nan nan nan nan nan nan]
[nan nan nan nan '11100011' nan nan]
[nan nan nan

[nan nan nan '11100001' nan '11111111' nan]
['11100011' nan nan nan '01110000' nan nan]
['11100011' nan nan nan nan nan nan]
[nan nan nan '11100001' nan '11111111' nan]
[nan nan nan nan '11111000' nan nan]
[nan nan nan '11100001' nan '11111111' nan]
[nan nan nan '11110001' nan nan nan]
['10001111' nan nan nan nan nan nan]
['01111100' nan nan nan '11111111' nan nan]
['01111100' nan nan nan nan nan nan]
['01111100' nan nan nan '11111111' nan nan]
['01111100' nan nan nan nan nan nan]
[nan nan nan '11111000' '11111111' nan nan]
[nan nan nan nan '11111111' nan nan]
['10001111' nan nan '01110000' nan nan nan]
['10001111' nan nan '11110001' nan nan nan]
['10001111' nan nan '11110001' nan nan nan]
['10001111' nan nan nan nan nan nan]
[nan nan nan '11110001' nan nan nan]
[nan nan nan '11110001' nan nan nan]
[nan nan nan '11100011' nan nan nan]
[nan nan '11111111' '11000111' nan nan nan]
['11100011' nan nan nan '01110000' nan nan]
['11000111' nan nan nan nan nan nan]
[nan nan '11111111' '1100011

[nan nan nan '11111000' nan nan nan]
['01110000' nan nan nan '11111000' nan nan]
['11111000' nan nan nan nan nan nan]
['11111000' '11111111' nan nan nan nan nan]
[nan nan nan nan '01110000' nan nan]
['11111000' nan nan nan nan nan nan]
[nan nan nan '11111000' nan nan nan]
['00011111' nan nan nan '11111111' nan nan]
['11110000' nan nan nan '11111111' nan nan]
['11111000' nan nan nan nan nan nan]
['11111000' nan nan nan '11111111' nan nan]
['11111000' nan nan nan nan nan nan]
[nan nan nan '01111100' nan nan nan]
[nan nan nan '01111100' nan nan nan]
[nan nan nan '01111100' nan nan nan]
[nan nan nan '01111100' nan nan nan]
[nan nan nan '01111100' nan nan nan]
[nan nan nan '01111100' nan nan nan]
['11111000' nan nan nan nan nan nan]
['11111000' nan nan nan nan nan nan]
[nan nan nan '11111000' nan nan nan]
[nan nan nan '11111000' nan nan nan]
['11111000' nan nan nan '11111000' nan nan]
['11111000' nan nan nan nan nan nan]
['11111000' nan nan nan nan nan nan]
['11111000' nan nan nan nan nan n

['11111111' nan nan nan nan nan nan]
['11111111' nan nan nan nan nan nan]
['00001111' nan nan nan nan nan nan]
['00011111' nan nan nan nan nan nan]
['10001111' nan nan nan nan nan nan]
['11111111' nan nan nan nan nan nan]
[nan nan nan nan '11111111' nan nan]
[nan nan nan nan '11111111' nan nan]
['11111111' nan nan nan '01110000' nan nan]
['11111111' nan nan '00001111' nan nan nan]
[nan nan nan '11111111' nan nan nan]
['10001111' nan nan nan nan nan nan]
['11111111' nan nan '00001110' nan nan nan]
['10001111' nan nan nan nan nan nan]
[nan nan nan '00001110' nan nan nan]
['11111111' nan nan nan '11111111' nan nan]
['00111110' nan nan nan nan nan nan]
['01111000' nan nan nan nan nan nan]
['01111000' nan nan nan nan nan nan]
['01111000' nan nan nan nan nan nan]
['01111100' nan nan nan '11111111' nan nan]
['11111111' nan nan nan nan nan nan]
['11111111' nan nan nan nan nan nan]
['00111110' nan nan nan nan nan nan]
['00111100' nan nan nan nan nan nan]
['00111100' nan nan nan nan nan nan]
[na

['11000001' nan nan nan '11111111' nan nan]
['11000001' nan nan nan nan nan nan]
['11000001' nan nan nan nan nan nan]
['10000011' nan nan nan nan nan nan]
['11000011' nan nan nan '11111111' nan nan]
['11000111' nan nan nan nan nan nan]
['11000011' nan nan nan nan nan nan]
['11000111' nan nan nan '11111111' nan nan]
['11000001' nan nan nan nan nan nan]
['11000111' nan nan nan '11111111' nan nan]
['10000111' nan nan nan '11111111' nan nan]
['11000001' nan nan nan nan nan nan]
['11000001' nan nan nan '10001111' nan nan]
['11000001' nan nan nan '11111111' nan nan]
['11000001' nan nan nan nan nan nan]
['11000011' nan nan nan '11111111' nan nan]
['11000001' nan nan nan '11111111' nan nan]
['11100011' nan nan nan nan nan nan]
['11000011' nan nan nan '11111111' nan nan]
['11000011' nan nan nan nan nan nan]
['11100011' nan nan nan nan nan nan]
['11000011' nan nan nan '11111111' nan nan]
['11000011' nan nan nan nan nan nan]
['11000011' nan nan nan '11000011' nan nan]
['11000011' nan nan nan nan 

...

[nan '11111111' nan nan nan '11111111' nan]
[nan '11111111' nan nan nan '11111111' nan]
[nan '11111111' nan nan nan '11111111' nan]
[nan '11111111' nan nan nan '11111111' nan]
[nan nan nan nan nan '11111111' nan]
[nan '11111111' nan nan nan '11111111' nan]
[nan nan nan nan nan '11111111' nan]
[nan nan nan nan nan '11111111' nan]
[nan '11111111' nan nan nan '11111111' nan]
[nan nan nan nan nan '11111111' '11111111']
[nan nan nan nan nan '11111111' '11111111']
[nan nan nan nan nan '11111111' '11111111']
[nan nan nan nan nan '11111111' '11111111']
[nan nan nan nan nan '11111111' '11111111']
[nan nan nan nan nan '11111111' '11111111']
[nan nan nan nan nan '11111111' '11111111']
[nan nan nan nan nan '11111111' '11111111']
[nan nan nan nan nan '11111111' '11111111']
[nan nan nan nan nan '11111111' '11111111']
[nan '11111111' nan nan nan '11111111' nan]
[nan '11111111' nan nan nan '11111111' nan]
[nan '11111111' nan nan nan '11111111' nan]
[nan '11111111' nan nan nan '11111111' nan]
[nan '111

[nan nan nan nan nan '11111111' '11111111']
[nan '11111111' nan nan nan '11111111' nan]
[nan '11111111' nan nan nan '11111111' nan]
[nan nan nan nan nan '11111111' nan]
[nan '11111111' nan nan nan '11111111' nan]
[nan nan nan nan nan '11111111' nan]
[nan nan nan nan nan '11111111' nan]
[nan nan nan nan nan '11111111' nan]
[nan nan nan nan nan '11111111' '11111111']
[nan nan nan nan nan '11111111' '11111111']
[nan nan nan nan nan '11111111' '11111111']
[nan nan nan nan nan '11111111' '11111111']
[nan nan nan nan nan '11111111' '11111111']
[nan nan nan nan nan '11111111' '11111111']
[nan '11111111' nan nan nan '11111111' nan]
[nan nan nan nan nan '11111111' '11111111']
[nan nan nan nan nan '11111111' '11111111']
[nan nan nan nan nan '11111111' '11111111']
[nan '11111111' nan nan nan '11111111' nan]
[nan '11111111' nan nan nan '11111111' nan]
[nan '11111111' nan nan nan '11111111' nan]
[nan '11111111' nan nan nan '11111111' nan]
[nan nan nan nan nan '11111111' '11111111']
[nan '11111111' 

[nan nan nan nan '11111111' nan nan]
[nan nan nan nan '11000111' nan nan]
['10000111' nan nan nan nan nan nan]
['10000111' nan nan nan nan nan nan]
['10000111' nan nan nan nan nan nan]
[nan nan nan nan '11111111' nan nan]
['11100011' nan nan nan nan nan nan]
['11110001' nan nan nan nan nan nan]
['11100011' nan nan nan nan nan nan]
['11100011' nan nan nan nan nan nan]
['11110001' nan nan nan nan nan nan]
['11110001' nan nan nan nan nan nan]
['01111100' nan nan nan nan nan nan]
['11110001' nan nan nan nan nan nan]
['11000111' nan nan nan nan nan nan]
['10001111' nan nan nan nan nan nan]
['10001111' nan nan nan nan nan nan]
['11111111' nan nan nan nan nan nan]
['11111111' nan nan nan nan nan nan]
['11111111' nan nan nan nan nan nan]
['11000111' nan nan nan nan nan nan]
[nan nan nan nan '11100111' nan nan]
['11000111' nan nan nan nan nan nan]
[nan nan nan nan '11111111' nan nan]
[nan nan nan nan '11000111' nan nan]
['10000111' nan nan nan nan nan nan]
['10000111' nan nan nan nan nan nan]
[

[nan nan nan '11000111' nan nan nan]
[nan nan nan nan '11111111' nan nan]
[nan nan nan '11111111' nan nan nan]
[nan nan nan '11111111' nan nan nan]
[nan nan nan '11111111' nan nan nan]
[nan nan nan '11111111' nan nan nan]
['11100011' nan nan nan '11111111' nan nan]
['10000011' nan nan nan nan nan nan]
[nan nan nan '11111111' '11000111' nan nan]
['11110011' nan nan nan '11110001' nan nan]
[nan nan nan '11111111' nan nan nan]
['11100011' nan nan nan nan nan nan]
['11110001' nan nan nan nan nan nan]
['11000111' nan nan nan '11000111' nan nan]
['11000011' nan nan nan nan nan nan]
['11000011' nan nan nan nan nan nan]
['11000011' nan nan nan nan nan nan]
['11000011' nan nan nan nan nan nan]
['11000011' nan nan nan nan nan nan]
['11111111' nan nan nan nan nan nan]
['11000011' nan nan nan nan nan nan]
['11000111' nan nan nan nan nan nan]
['10000111' nan nan nan nan nan nan]
[nan nan nan '11111111' nan nan nan]
[nan nan nan '11111111' nan nan nan]
[nan nan nan '01111100' nan nan nan]
[nan nan n

['10000011' nan nan nan nan nan nan]
['11000111' nan nan nan nan nan nan]
['11000011' nan nan nan nan nan nan]
['11000111' nan nan nan nan nan nan]
[nan nan nan nan '10001111' nan nan]
['11000111' nan nan nan nan nan nan]
[nan nan nan nan '10001111' nan nan]
['11000111' nan nan nan nan nan nan]
[nan nan nan nan '10001111' nan nan]
[nan nan nan '10001111' nan nan nan]
['00011111' nan nan nan nan nan nan]
[nan nan nan '10000011' nan nan nan]
[nan nan nan '00000111' nan nan nan]
['11000111' nan nan nan '11111111' nan nan]
['11000111' nan nan nan nan nan nan]
['10000011' nan nan nan nan nan nan]
['10001111' nan nan nan '11000111' nan nan]
['10000111' nan nan nan nan nan nan]
[nan nan nan nan '11111111' nan nan]
['10000111' nan nan nan nan nan nan]
['11110001' nan nan nan '11111111' nan nan]
['11000011' nan nan nan nan nan nan]
[nan nan nan nan '11100111' nan nan]
[nan nan nan nan '11000011' nan nan]
['11100001' nan nan nan nan nan nan]
['11100001' nan nan nan nan nan nan]
['11100001' nan n

**I wonder if it isn't better to have these `cause` columns combined into one.**

**Same with the wind or `aspect` columns. Then we could groupby what the cause was to then analyze aspect or something like that. Or, like above, it only looks like each column is represented once. So I could take the only available value, and then split that into 8 new columns representing each direction, and fill with the available value. And then drop these MULTI columns.**

**Actually, same with the avalanche problems?**

# Correlation analysis

In [4]:
features_and_labels = features = pd.concat([labeled_data.data, labeled_data.label], axis=1)
features_and_labels.shape

(13115, 314)

In [5]:
# drop columns that include text, create correlations matrix
only_numeric = features_and_labels.select_dtypes(include='number')
correlations = only_numeric.corr()

# shorten the x axis to only the labels
keep_cols = [col for col in labeled_data.label if col in correlations.columns]
drop_rows = list(set(correlations.columns) - set(keep_cols))
correlations = correlations.loc[drop_rows, keep_cols]

# there are some rows with only nan values, we can drop those here
empty_rows = [item for item in correlations.index if correlations.loc[item].isna().all() == True]
correlations.drop(empty_rows, inplace = True)

In [6]:
print(correlations.shape)

(165, 55)


**That is a lot of columns, still. Another thing to try would be to drop all rows where the absolute value of the correlation is below 0.25 or 0.25, for all values. These input features probably don't need to be considered.**

In [7]:
weak_correlations = [(abs(correlations.loc[idx, :].values) < 0.25).all() for idx in correlations.index]
correlations.loc[weak_correlations, :]

CLASS_danger_level  CLASS_emergency_warning  \
wind_chg_start_12_0                        0.127779                 0.056323   
wind_dir_N_0                              -0.035648                -0.048012   
precip_most_exposed_0                      0.006142                 0.002346   
wind_dir_SE_0                             -0.174427                -0.092408   
wind_chg_start_18_0                        0.117878                 0.039247   
problem_new-loose_lev_fill_4_1            -0.004446                -0.009574   
temp_fl_start_0_0                         -0.024942                -0.011614   
problem_new-loose_lev_fill_2_1             0.024787                -0.011548   
wind_chg_dir_SE_0                          0.015820                -0.030780   
wind_chg_dir_SW_0                          0.137233                 0.100253   
wind_chg_dir_S_0                           0.028854                -0.020837   
wind_chg_start_0_0                         0.037859                 0.020968   
wind_dir_S_0                              -0.072425                -0.024265   
wind_dir_E_0                              -0.064153                -0.065905   
wind_chg_dir_W_0                           0.135825                 0.088633   
wind_chg_dir_NW_0                          0.103763                 0.063405   
temp_fl_start_6_0                          0.066676                 0.019519   
problem_new-slab_lev_fill_4_1              0.010713                -0.002885   
problem_wet-slab_lev_fill_2_1              0.165316                 0.128492   
wind_chg_dir_N_0                           0.043717                 0.004366   
wind_dir_SW_0                              0.088017                 0.080403   
precip_0                                   0.001479                -0.001473   
problem_wet-loose_lev_fill_3_1            -0.037891                -0.008163   
problem_pwl-slab_lev_fill_3_1             -0.001340                -0.002885   
wind_dir_NW_0                              0.039033                -0.014240   
problem_new-slab_lev_fill_2_1              0.004638                -0.004998   
temp_fl_start_12_0                         0.069564                 0.003476   
wind_dir_W_0                               0.194684                 0.146047   
problem_pwl-slab_lev_fill_2_1              0.015460                -0.017940   
temp_fl_start_18_0                        -0.028266                 0.025599   
problem_drift-slab_lev_fill_4_1            0.002095                 0.019883   
wind_chg_dir_E_0                           0.006003                -0.028183   
wind_chg_dir_NE_0                          0.031747                -0.011310   
wind_dir_NE_0                             -0.054276                -0.054734   
problem_glide_lev_fill_3_1                -0.001340                -0.002885   
wind_chg_start_6_0                         0.101921                 0.035072   

                                 CLASS_problem_1  CLASS_problem_2  \
wind_chg_start_12_0                    -0.058495         0.039847   
wind_dir_N_0                            0.005104        -0.043055   
precip_most_exposed_0                   0.000215         0.017305   
wind_dir_SE_0                          -0.072229        -0.091066   
wind_chg_start_18_0                    -0.068729         0.034206   
problem_new-loose_lev_fill_4_1         -0.012899        -0.009626   
temp_fl_start_0_0                      -0.056879        -0.012860   
problem_new-loose_lev_fill_2_1         -0.015028         0.000360   
wind_chg_dir_SE_0                      -0.090891        -0.018517   
wind_chg_dir_SW_0                       0.015269         0.086503   
wind_chg_dir_S_0                       -0.024049         0.026867   
wind_chg_start_0_0                      0.005663         0.002809   
wind_dir_S_0                           -0.024947        -0.034335   
wind_dir_E_0                           -0.010206        -0.051797   
wind_chg_dir_W_0                     

In [8]:
drop_weak = correlations.loc[weak_correlations, :].index.values
correlations.drop(drop_weak, inplace=True)
correlations.shape

(129, 55)

In [9]:
fig, ax = plt.subplots(figsize=(20, 30))
sns.heatmap(correlations, xticklabels=correlations.columns, yticklabels=correlations.index, cmap='RdYlGn', ax=ax)
ax.set_title('Correlation Matrix for the Training Set Features')

plt.show()

**So according to our correlation analysis, there are about 129 features out of the original 291 that are useful in predicting one or more of our labels. We can save those below and compare them with what features our other analysis find useful.**

In [14]:
keep_columns_correlations = list(correlations.index)
keep_columns_correlations

['problem_wet-slab_trig_1',
 'problem_drift-slab_dsize_1',
 'problem_wet-slab_prob_1',
 'problem_drift-slab_cause_new-snow_1',
 'problem_new-slab_lev_fill_1_1',
 'problem_drift-slab_prob_1',
 'problem_amount_1',
 'problem_new-slab_trig_1',
 'problem_drift-slab_aspect_SE_1',
 'temp_min_0',
 'problem_glide_aspect_SE_1',
 'problem_drift-slab_lev_max_1',
 'problem_pwl-slab_aspect_W_1',
 'problem_pwl-slab_lev_max_1',
 'problem_drift-slab_lev_min_1',
 'problem_new-loose_trig_1',
 'problem_glide_trig_1',
 'problem_wet-slab_aspect_W_1',
 'problem_pwl-slab_aspect_NE_1',
 'problem_wet-loose_dsize_1',
 'problem_wet-loose_prob_1',
 'problem_glide_aspect_S_1',
 'problem_new-loose_aspect_N_1',
 'problem_wet-loose_aspect_E_1',
 'problem_wet-loose_aspect_S_1',
 'problem_wet-loose_aspect_SW_1',
 'problem_pwl-slab_prob_1',
 'problem_drift-slab_aspect_N_1',
 'problem_new-loose_aspect_SW_1',
 'problem_wet-slab_lev_max_1',
 'problem_wet-slab_lev_fill_1_1',
 'problem_glide_dsize_1',
 'problem_new-loose_aspe

# Feature importances

Next, we can look at which features a Random Forest model thinks are important. To do this, we can use the whole dataset as a predictor for the labels.

This will look something like:
```python
X, y = labeled_data.data, labels

num_iter = 10
random = 0   #makes sure the models and grid search object give the same results every time

#instantiate models here
forest = RandomForestRegressor(random_state=random)

#define parameters dictionaries here
ensemble_params = {'n_estimators': [1, 2, 4, 8, 16, 32, 64, 100, 200],
             'max_depth': [1, 2, 4, 6, 8, 10, 20, 30, None],  #higher values often lead to overfitting
             'max_features': ['auto', 'sqrt', 'log2'],
             'min_samples_leaf': [1, 2, 4],
             'min_samples_split': [2, 5, 10],
            'bootstrap': [True, False]
            }

#create our random search cv objects
forest_search = RandomizedSearchCV(estimator=forest,
                        param_distributions=ensemble_params,
                        n_iter=num_iter,         #number of parameter settings sampled
                        scoring='neg_root_mean_squared_error',
                        n_jobs=-1,         #use all processors available
                        cv=5,
                        verbose=2,         #print results during tuning
                        random_state=random
)

#now fit the search objects and return them
forest_model = forest_search.fit(X, y)


print('Best RandomForest model:')
print(forest_model.best_params_, '\n')
```

```python
def plot_importances(df, labels, rf):
    """
    Input a dataframe as well as trained RandomForest model, plot feature importances.
    
    """
    columns = df.columns #all columns except the target column
    importances_rf = rf.best_estimator_.feature_importances_
    
    #take the top ten feature importances
    idx_rf = np.argsort(importances_rf)
    #idx_rf = np.argsort(importances_rf)[-10:]
    
    fig, ax = plt.subplots(figsize=(12, 7))
    #random forest and xgboost feature importances on the same plot
    ax.set_title('Feature Importances')
    ax.barh(range(len(idx_rf)), importances_rf[idx_rf], color='tab:blue', align='center', alpha=0.5, label='RF')

    ax.set_yticks(range(len(idx_rf)))
    ax.set_yticklabels([columns[i] for i in idx_rf], fontsize=10)
    ax.set_xlabel('Relative Importance')

    ax.legend()
    plt.show()
    
# need to make sure this is using the "best" model by default
plot_importances(labeled_data.data, labels, forest_model) 

```

# PCA
- https://scentellegher.github.io/machine-learning/2020/01/27/pca-loadings-sklearn.html
- https://stackoverflow.com/questions/22984335/recovering-features-names-of-explained-variance-ratio-in-pca-with-sklearn

Look here for previous work that I've done as a guide: https://github.com/Jakidxav/udacity_ml_pytorch/blob/master/customer_segments_clustering/arvato_pca_clustering.ipynb